In [285]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Load the processed dataset
df = pd.read_csv('Dataset/train_processed.csv')
test_df = pd.read_csv('Dataset/test_processed.csv')

bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)
bool_cols_test = test_df.select_dtypes(include='bool').columns
test_df[bool_cols_test] = test_df[bool_cols_test].astype(int)

In [286]:
# Compute pair frequency (A → B)
account_pairs = df.groupby(['From Account', 'To Account']).size().reset_index(name='pair_frequency')

# Compute reverse pair frequency (B → A)
reverse_pairs = account_pairs.copy()
reverse_pairs.columns = ['To Account', 'From Account', 'reverse_pair_frequency']  # flip column names

# Merge both into the original dataframe
df = df.merge(account_pairs, on=['From Account', 'To Account'], how='left')
df = df.merge(reverse_pairs, on=['From Account', 'To Account'], how='left')  # correct merge

# Fill missing frequencies
df['pair_frequency'].fillna(1, inplace=True)
df['reverse_pair_frequency'].fillna(0, inplace=True)

# Flag circular transactions
df['is_circular'] = (df['reverse_pair_frequency'] > 0).astype(int)

C:\Users\emanu\AppData\Local\Temp\ipykernel_23904\1452891232.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['pair_frequency'].fillna(1, inplace=True)
C:\Users\emanu\AppData\Local\Temp\ipykernel_23904\1452891232.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

In [287]:
# Compute pair frequency (A → B)
account_pairs = test_df.groupby(['From Account', 'To Account']).size().reset_index(name='pair_frequency')

# Compute reverse pair frequency (B → A)
reverse_pairs = account_pairs.copy()
reverse_pairs.columns = ['To Account', 'From Account', 'reverse_pair_frequency']  # flip column names

# Merge both into the original dataframe
test_df = test_df.merge(account_pairs, on=['From Account', 'To Account'], how='left')
test_df = test_df.merge(reverse_pairs, on=['From Account', 'To Account'], how='left')  # correct merge

# Fill missing frequencies
test_df['pair_frequency'].fillna(1, inplace=True)
test_df['reverse_pair_frequency'].fillna(0, inplace=True)

# Flag circular transactions
test_df['is_circular'] = (test_df['reverse_pair_frequency'] > 0).astype(int)

C:\Users\emanu\AppData\Local\Temp\ipykernel_23904\3936049671.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['pair_frequency'].fillna(1, inplace=True)
C:\Users\emanu\AppData\Local\Temp\ipykernel_23904\3936049671.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

In [288]:
import  networkx as nx
# Initialize a directed graph
G = nx.DiGraph()

# Add edges with attributes
for idx, row in df.iterrows():
    G.add_edge(
        row['From Account'],
        row['To Account'],
        amount_log=row['Amount Paid_log'],
        pair_frequency=row['pair_frequency'],
        reverse_pair_frequency=row['reverse_pair_frequency'],
        is_circular=row['is_circular']
    )


In [289]:
import  networkx as nx
# Initialize a directed graph
G_test = nx.DiGraph()

# Add edges with attributes
for idx, row in test_df.iterrows():
    G_test.add_edge(
        row['From Account'],
        row['To Account'],
        amount_log=row['Amount Paid_log'],
        pair_frequency=row['pair_frequency'],
        reverse_pair_frequency=row['reverse_pair_frequency'],
        is_circular=row['is_circular']
    )

In [290]:
#in_degree_centrality = nx.in_degree_centrality(G)
#out_degree_centrality = nx.out_degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G, k=min(1000, len(G.nodes())))

In [291]:
#in_degree_centrality_test = nx.in_degree_centrality(G_test)
#out_degree_centrality_test = nx.out_degree_centrality(G_test)
betweenness_centrality_test = nx.betweenness_centrality(G_test, k=min(1000, len(G_test.nodes())))

In [292]:
#df['from_out_degree_centrality'] = df['From Account'].map(out_degree_centrality).fillna(0)
df['from_betweenness_centrality'] = df['From Account'].map(betweenness_centrality).fillna(0)
#df['to_in_degree_centrality'] = df['To Account'].map(in_degree_centrality).fillna(0)
df['to_betweenness_centrality'] = df['To Account'].map(betweenness_centrality).fillna(0)

In [293]:
#test_df['from_out_degree_centrality'] = test_df['From Account'].map(out_degree_centrality_test).fillna(0)
test_df['from_betweenness_centrality'] = test_df['From Account'].map(betweenness_centrality_test).fillna(0)
#test_df['to_in_degree_centrality'] = test_df['To Account'].map(in_degree_centrality_test).fillna(0)
test_df['to_betweenness_centrality'] = test_df['To Account'].map(betweenness_centrality_test).fillna(0)

In [294]:
!pip install node2vec

  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached networkx-3.5-py3-none-any.whl (2.0 MB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: networkx
    Found existing installation: networkx 2.6.3
    Uninstalling networkx-2.6.3:
      Successfully uninstalled networkx-2.6.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
karateclub 1.3.3 requires networkx<2.7, but you have networkx 3.5 which is incompatible.
karateclub 1.3.3 requires numpy<1.23.0, but you have numpy 1.26.4 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [295]:
df = df.drop(columns=['From Account', 'To Account'], errors='ignore')

# Separate features and target
X = df.drop(columns=['Is Laundering'])
y = df['Is Laundering']

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [296]:
#!pip install optuna

In [297]:
test_df = test_df.drop(columns=['From Account', 'To Account'], errors='ignore')

In [298]:
from sklearn.metrics import roc_auc_score, confusion_matrix

def calculate_balanced_accuracy(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
    return (tpr + tnr) / 2

def calculate_fraud_capture_rate(y_true, y_prob, N=485):
    sorted_indices = np.argsort(y_prob)[::-1]
    top_N_indices = sorted_indices[:N]
    frauds_in_top_N = np.sum(y_true.iloc[top_N_indices] if hasattr(y_true, 'iloc') else y_true[top_N_indices])
    total_frauds = np.sum(y_true)
    return frauds_in_top_N / total_frauds if total_frauds > 0 else 0

def calculate_composite_score(y_true, y_pred, y_prob, N=485):
    auc_score = roc_auc_score(y_true, y_prob)
    balanced_acc = calculate_balanced_accuracy(y_true, y_pred)
    fraud_capture = calculate_fraud_capture_rate(y_true, y_prob, N)
    return (auc_score + balanced_acc + fraud_capture) / 3


### LGBM

In [299]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, confusion_matrix
import numpy as np

def objective(trial):
    scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
    
    # 🔧 Threshold being tuned by Optuna
    threshold = trial.suggest_float("threshold", 0.05, 0.5)

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.2),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'num_leaves': trial.suggest_int('num_leaves', 20, 40),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 5),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'max_bin': trial.suggest_int('max_bin', 200, 300),
        'scale_pos_weight': scale_pos_weight,
        'force_col_wise': True,
        'verbosity': -1
    }

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    auc_scores = [] 
    balanced_accuracies = []
    fraud_capture_rates = []
    scores = []

    for train_idx, val_idx in kf.split(X_train, y_train):
        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model = lgb.LGBMClassifier(**params, random_state=42)
        model.fit(X_fold_train, y_fold_train)

        y_prob = model.predict_proba(X_fold_val)[:, 1]
        y_pred = (y_prob >= threshold).astype(int)

        auc_score = roc_auc_score(y_fold_val, y_prob)
        balanced_acc = calculate_balanced_accuracy(y_fold_val, y_pred)
        fraud_capture = calculate_fraud_capture_rate(y_fold_val, y_prob, N=485)
        score = calculate_composite_score(y_fold_val, y_pred, y_prob, N=485)

        auc_scores.append(auc_score)
        balanced_accuracies.append(balanced_acc)    
        fraud_capture_rates.append(fraud_capture)
        scores.append(score)

        print(f"AUC: {np.mean(auc_scores)}, Balanced Accuracy: {np.mean(balanced_accuracies)}, Fraud Capture Rate: {np.mean(fraud_capture_rates)}, Composite Score: {np.mean(scores)}")

    return np.mean(scores)


In [300]:
import optuna

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)

print("Best Hyperparameters:", study.best_params)
print("Best Composite Score:", round(study.best_value, 4))


[I 2025-06-16 06:26:09,975] A new study created in memory with name: no-name-762f8d1d-6020-492b-94b2-7c00ef5d1e85
C:\Users\emanu\AppData\Local\Temp\ipykernel_23904\3458708873.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.2),


AUC: 0.9910975584153844, Balanced Accuracy: 0.9908256880733946, Fraud Capture Rate: 0.981651376146789, Composite Score: 0.9878582075451893
AUC: 0.9955487792076922, Balanced Accuracy: 0.9953842366445471, Fraud Capture Rate: 0.9908256880733946, Composite Score: 0.9939195679752112
AUC: 0.9970241973213513, Balanced Accuracy: 0.9954076729145466, Fraud Capture Rate: 0.9938837920489297, Composite Score: 0.9954385540949424
AUC: 0.9977681479910134, Balanced Accuracy: 0.99655575468591, Fraud Capture Rate: 0.9954128440366973, Composite Score: 0.9965789155712068


[I 2025-06-16 06:26:11,445] Trial 0 finished with value: 0.9972631324569654 and parameters: {'threshold': 0.21854305348131314, 'n_estimators': 291, 'learning_rate': 0.0483437145318464, 'max_depth': 11, 'num_leaves': 23, 'min_child_samples': 16, 'subsample': 0.7174250836504598, 'subsample_freq': 5, 'colsample_bytree': 0.8404460046972835, 'reg_alpha': 0.7080725777960455, 'reg_lambda': 0.020584494295802447, 'max_bin': 297}. Best is trial 0 with value: 0.9972631324569654.


AUC: 0.9982145183928107, Balanced Accuracy: 0.997244603748728, Fraud Capture Rate: 0.9963302752293579, Composite Score: 0.9972631324569654
AUC: 0.9853342220729326, Balanced Accuracy: 0.5, Fraud Capture Rate: 0.981651376146789, Composite Score: 0.8223285327399071


C:\Users\emanu\AppData\Local\Temp\ipykernel_23904\3458708873.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.2),


...

AUC: 0.9913967207608171, Balanced Accuracy: 0.9908256880733946, Fraud Capture Rate: 0.981651376146789, Composite Score: 0.9879579283270002
AUC: 0.9956983603804086, Balanced Accuracy: 0.9953842366445471, Fraud Capture Rate: 0.9908256880733946, Composite Score: 0.9939694283661167
AUC: 0.9971259986407729, Balanced Accuracy: 0.9954076729145466, Fraud Capture Rate: 0.9938837920489297, Composite Score: 0.9954724878680831
AUC: 0.9978444989805797, Balanced Accuracy: 0.99655575468591, Fraud Capture Rate: 0.9954128440366973, Composite Score: 0.9966043659010623
AUC: 0.9982755991844637, Balanced Accuracy: 0.997244603748728, Fraud Capture Rate: 0.9963302752293579, Composite Score: 0.9972834927208499


[I 2025-06-16 06:27:06,875] Trial 48 finished with value: 0.9972834927208499 and parameters: {'threshold': 0.36156474593486915, 'n_estimators': 194, 'learning_rate': 0.03735763055265818, 'max_depth': 9, 'num_leaves': 37, 'min_child_samples': 16, 'subsample': 0.7185447094803018, 'subsample_freq': 4, 'colsample_bytree': 0.9140141519954941, 'reg_alpha': 0.5819963477994207, 'reg_lambda': 0.3560513482570773, 'max_bin': 239}. Best is trial 29 with value: 0.9976432996466595.
C:\Users\emanu\AppData\Local\Temp\ipykernel_23904\3458708873.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.2),


AUC: 0.9934142296307182, Balanced Accuracy: 0.9908256880733946, Fraud Capture Rate: 0.981651376146789, Composite Score: 0.9886304312836339
AUC: 0.9967071148153591, Balanced Accuracy: 0.9953842366445471, Fraud Capture Rate: 0.9908256880733946, Composite Score: 0.9943056798444335
AUC: 0.9977686805583169, Balanced Accuracy: 0.9954076729145466, Fraud Capture Rate: 0.9938837920489297, Composite Score: 0.9956867151739309
AUC: 0.9983265104187377, Balanced Accuracy: 0.9965414509898349, Fraud Capture Rate: 0.9954128440366973, Composite Score: 0.9967602684817565


[I 2025-06-16 06:27:07,718] Trial 49 finished with value: 0.9974082147854052 and parameters: {'threshold': 0.398248483997963, 'n_estimators': 140, 'learning_rate': 0.19956541865610614, 'max_depth': 8, 'num_leaves': 33, 'min_child_samples': 38, 'subsample': 0.7876637928809842, 'subsample_freq': 5, 'colsample_bytree': 0.7558558252407594, 'reg_alpha': 0.33007796536303924, 'reg_lambda': 0.5254810758337607, 'max_bin': 268}. Best is trial 29 with value: 0.9976432996466595.


AUC: 0.9986612083349901, Balanced Accuracy: 0.997233160791868, Fraud Capture Rate: 0.9963302752293579, Composite Score: 0.9974082147854052
Best Hyperparameters: {'threshold': 0.47035236108226064, 'n_estimators': 149, 'learning_rate': 0.11712570986073118, 'max_depth': 10, 'num_leaves': 31, 'min_child_samples': 43, 'subsample': 0.7708187394233096, 'subsample_freq': 5, 'colsample_bytree': 0.8494618593704473, 'reg_alpha': 0.18396654220854464, 'reg_lambda': 0.10440040123828889, 'max_bin': 296}
Best Composite Score: 0.9976


In [301]:
import lightgbm as lgb
import pandas as pd
import numpy as np

# ✅ Best hyperparameters from Optuna
best_params_lgb = {
    'n_estimators': 171,
    'learning_rate': 0.12792961140536083,
    'max_depth': 10,
    'num_leaves': 34,
    'min_child_samples': 50,
    'subsample': 0.9208777758688815,
    'subsample_freq': 4,
    'colsample_bytree': 0.7677410569512204,
    'reg_alpha': 0.28067157957490324,
    'reg_lambda': 0.0859139708073734,
    'max_bin': 232,
    'scale_pos_weight': len(y_train[y_train == 0]) / len(y_train[y_train == 1]),  # Handling class imbalance
    'force_col_wise': True,
    'verbosity': -1
}

# 🔁 Train final model with the best parameters
final_lgb = lgb.LGBMClassifier(**best_params_lgb, random_state=42)
final_lgb.fit(X_train, y_train)

# 📊 Predict probabilities on test set
test_probs_lgb = final_lgb.predict_proba(test_df)[:, 1]

# 🔎 Apply the optimized threshold
best_threshold = 0.44286465058062224
test_preds = (test_probs_lgb >= best_threshold).astype(int)

# 💾 Save predictions in the required format
submission = pd.DataFrame({
    0: test_probs_lgb,
    1: test_preds
})
submission.to_csv("submission.txt", sep=" ", header=False, index=False)


In [302]:
with open("submission_lightgbm.txt", "w") as f:
    for prob, pred in zip(test_probs_lgb, test_preds):
        f.write(f"{prob} {pred}\n")


### XGB

In [303]:
import xgboost as xgb

def objective(trial):
    scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
    
    # Tune threshold as part of objective
    threshold = trial.suggest_float("threshold", 0.05, 0.5)

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.2),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.0, 5.0),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'scale_pos_weight': scale_pos_weight,
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'use_label_encoder': False,
        'verbosity': 0
    }

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    auc_scores = [] 
    balanced_accuracies = []
    fraud_capture_rates = []
    scores = []

    for train_idx, val_idx in kf.split(X_train, y_train):
        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model = xgb.XGBClassifier(**params)
        model.fit(X_fold_train, y_fold_train)

        y_prob = model.predict_proba(X_fold_val)[:, 1]
        y_pred = (y_prob >= threshold).astype(int)

        auc_score = roc_auc_score(y_fold_val, y_prob)
        balanced_acc = calculate_balanced_accuracy(y_fold_val, y_pred)
        fraud_capture = calculate_fraud_capture_rate(y_fold_val, y_prob, N=485)
        score = calculate_composite_score(y_fold_val, y_pred, y_prob, N=485)

        auc_scores.append(auc_score)
        balanced_accuracies.append(balanced_acc)    
        fraud_capture_rates.append(fraud_capture)
        scores.append(score)

        print(f"AUC: {np.mean(auc_scores)}, Balanced Accuracy: {np.mean(balanced_accuracies)}, Fraud Capture Rate: {np.mean(fraud_capture_rates)}, Composite Score: {np.mean(scores)}")

    return np.mean(scores)


In [304]:
import optuna

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)

print("✅ Best XGBoost Parameters:", study.best_params)
print("🎯 Best Composite Score:", round(study.best_value, 4))


[I 2025-06-16 06:27:08,125] A new study created in memory with name: no-name-0b7cc941-59c3-4e43-b351-7a8e45b7a11b
C:\Users\emanu\AppData\Local\Temp\ipykernel_23904\342457772.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.2),


AUC: 0.9911279994961476, Balanced Accuracy: 0.9908256880733946, Fraud Capture Rate: 0.981651376146789, Composite Score: 0.9878683545721104
AUC: 0.9955639997480739, Balanced Accuracy: 0.9953556292523971, Fraud Capture Rate: 0.9908256880733946, Composite Score: 0.9939151056912885
AUC: 0.9969587514817423, Balanced Accuracy: 0.9953886013197799, Fraud Capture Rate: 0.9938837920489297, Composite Score: 0.9954103816168173
AUC: 0.9977190636113067, Balanced Accuracy: 0.9965414509898349, Fraud Capture Rate: 0.9954128440366973, Composite Score: 0.9965577862126129


[I 2025-06-16 06:27:10,424] Trial 0 finished with value: 0.9972462289700903 and parameters: {'threshold': 0.21854305348131314, 'n_estimators': 291, 'learning_rate': 0.0483437145318464, 'max_depth': 11, 'min_child_weight': 2, 'gamma': 0.7799726016810132, 'subsample': 0.6232334448672797, 'colsample_bytree': 0.9464704583099741, 'reg_alpha': 0.6011150117432088, 'reg_lambda': 0.7080725777960455}. Best is trial 0 with value: 0.9972462289700903.


AUC: 0.9981752508890454, Balanced Accuracy: 0.997233160791868, Fraud Capture Rate: 0.9963302752293579, Composite Score: 0.9972462289700903


C:\Users\emanu\AppData\Local\Temp\ipykernel_23904\342457772.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.2),


...

AUC: 0.9985399592761877, Balanced Accuracy: 0.9970958505465985, Fraud Capture Rate: 0.9963302752293579, Composite Score: 0.9973220283507146


C:\Users\emanu\AppData\Local\Temp\ipykernel_23904\342457772.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.2),


AUC: 0.9924443138160519, Balanced Accuracy: 0.990711271597422, Fraud Capture Rate: 0.981651376146789, Composite Score: 0.9882689871867543
AUC: 0.996222156908026, Balanced Accuracy: 0.9952412062301105, Fraud Capture Rate: 0.9908256880733946, Composite Score: 0.9940963504038436
AUC: 0.9974424278584793, Balanced Accuracy: 0.9952932477101554, Fraud Capture Rate: 0.9938837920489297, Composite Score: 0.9955398225391882
AUC: 0.9980818208938594, Balanced Accuracy: 0.9964699357826166, Fraud Capture Rate: 0.9954128440366973, Composite Score: 0.9966548669043911


[I 2025-06-16 06:28:59,356] Trial 49 finished with value: 0.9973238935235129 and parameters: {'threshold': 0.3697889950333949, 'n_estimators': 204, 'learning_rate': 0.16000132648131124, 'max_depth': 13, 'min_child_weight': 4, 'gamma': 2.393649439733381, 'subsample': 0.631467877958743, 'colsample_bytree': 0.9183612418104717, 'reg_alpha': 0.20145218184243102, 'reg_lambda': 0.8953029173492731}. Best is trial 41 with value: 0.9974632828890261.


AUC: 0.9984654567150877, Balanced Accuracy: 0.9971759486260933, Fraud Capture Rate: 0.9963302752293579, Composite Score: 0.9973238935235129
✅ Best XGBoost Parameters: {'threshold': 0.31307056336795847, 'n_estimators': 178, 'learning_rate': 0.11822929101440706, 'max_depth': 14, 'min_child_weight': 6, 'gamma': 0.3549711252470789, 'subsample': 0.664864650758815, 'colsample_bytree': 0.9000304616249366, 'reg_alpha': 0.13243647978425693, 'reg_lambda': 0.826386408348063}
🎯 Best Composite Score: 0.9975


In [305]:
import xgboost as xgb
import pandas as pd
import numpy as np

# ✅ Best XGBoost hyperparameters from Optuna
best_params_xgb = {
    'n_estimators': 286,
    'learning_rate': 0.03441730150512816,
    'max_depth': 12,
    'min_child_weight': 4,
    'gamma': 0.06655413271247834,
    'subsample': 0.6364784353029371,
    'colsample_bytree': 0.960836926077814,
    'reg_alpha': 0.8384948295032638,
    'reg_lambda': 0.629887207892041,
    'scale_pos_weight': len(y_train[y_train == 0]) / len(y_train[y_train == 1]),  # Handling class imbalance
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'use_label_encoder': False,
    'verbosity': 0
}

# 🔁 Train the final XGBoost model on the full training set
final_xgb = xgb.XGBClassifier(**best_params_xgb, random_state=42)
final_xgb.fit(X_train, y_train)

# 📊 Predict probabilities on test set
test_probs_xgb = final_xgb.predict_proba(test_df)[:, 1]

# 🔎 Apply optimized classification threshold
best_threshold = 0.3595320880199634
test_preds = (test_probs_xgb >= best_threshold).astype(int)

# 💾 Save predictions to submission file
submission = pd.DataFrame({
    0: test_probs_xgb,
    1: test_preds
})
submission.to_csv("submission.txt", sep=" ", header=False, index=False)



In [306]:
with open("submission_xgb.txt", "w") as f:
    for prob, pred in zip(test_probs_xgb, test_preds):
        f.write(f"{prob} {pred}\n")


In [307]:
# Simple average ensemble
blended_probs = 0.35 * test_probs_lgb + 0.65 * test_probs_xgb

In [308]:
with open("submission_ens.txt", "w") as f:
    for prob, pred in zip(blended_probs, test_preds):
        f.write(f"{prob} {pred}\n")


In [309]:
# check if 0 and 1 in lgb and xgb predictions match
lgb_preds = (test_probs_lgb >= best_threshold).astype(int)
xgb_preds = (test_probs_xgb >= best_threshold).astype(int)
# show when they don't match
mismatch_indices = np.where(lgb_preds != xgb_preds)[0]
print("Indices where LGBM and XGBoost predictions mismatch:", mismatch_indices)
# for those get more info
for idx in mismatch_indices:
    print(f"Index: {idx}, LGBM Prob: {test_probs_lgb[idx]}, XGBoost Prob: {test_probs_xgb[idx]}, LGBM Pred: {lgb_preds[idx]}, XGBoost Pred: {xgb_preds[idx]}")

Indices where LGBM and XGBoost predictions mismatch: [11286 14461 15892 16898 17488]
Index: 11286, LGBM Prob: 0.6649797088567638, XGBoost Prob: 0.13041166961193085, LGBM Pred: 1, XGBoost Pred: 0
Index: 14461, LGBM Prob: 0.21828313666491841, XGBoost Prob: 0.7525576949119568, LGBM Pred: 0, XGBoost Pred: 1
Index: 15892, LGBM Prob: 0.1438838850416516, XGBoost Prob: 0.748516857624054, LGBM Pred: 0, XGBoost Pred: 1
Index: 16898, LGBM Prob: 0.47988928041974177, XGBoost Prob: 0.16566811501979828, LGBM Pred: 1, XGBoost Pred: 0
Index: 17488, LGBM Prob: 0.33708798914442784, XGBoost Prob: 0.4972667694091797, LGBM Pred: 0, XGBoost Pred: 1


In [310]:
# which one did I use on the submission?
# I used the XGBoost predictions for the final submission
# Save the final blended predictions
submission = pd.DataFrame({
    0: blended_probs,
    1: (blended_probs >= best_threshold).astype(int)
})

submission.to_csv("submission_final.txt", sep=" ", header=False, index=False)

In [311]:
test_df = pd.read_csv('Dataset/test_processed.csv')
same_account_df = test_df[test_df['From Account'] == test_df['To Account']]

# Count how many such transactions exist
count_same = same_account_df.shape[0]
total = test_df.shape[0]

print(f"Same-account transactions: {count_same} out of {total} ({(count_same / total) * 100:.4f}%)")
# save index for those
same_account_indices = same_account_df.index.tolist()
# see how we predicted them
same_account_probs = blended_probs[same_account_indices]
same_account_probs

Same-account transactions: 40 out of 23743 (0.1685%)


array([6.07359200e-04, 5.97210359e-04, 9.49800903e-05, 7.54981269e-05,
       2.26995266e-04, 7.13328341e-05, 3.77053041e-04, 5.81680185e-05,
       5.95065390e-04, 1.65088917e-04, 6.02999262e-04, 7.27311202e-05,
       6.59180268e-05, 9.34322722e-05, 1.13515587e-03, 1.04932997e-04,
       3.89213893e-04, 7.27775641e-05, 1.07238620e-04, 6.19921390e-05,
       2.70240539e-03, 8.48505224e-05, 8.48505224e-05, 7.89238663e-05,
       6.20597582e-05, 8.82540021e-05, 7.80190057e-05, 3.45947462e-03,
       3.18787582e-04, 8.15105546e-05, 5.44862995e-04, 3.45270261e-03,
       8.51612117e-05, 8.17038311e-05, 7.27401813e-05, 1.12644266e-03,
       5.82439770e-05, 2.20672875e-04, 5.03998341e-05, 1.04932997e-04])

All corrected